In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [10]:
X = np.loadtxt('feature_data.csv', delimiter=',')
y = np.loadtxt('class_data.csv', delimiter=',')


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.3, random_state=42)

In [23]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
total = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]: total +=1 
        
print("Accuracy: {}".format(total / len(y_pred)))

Accuracy: 0.6028708133971292
